In [0]:
import torch
from IPython.display import Image, clear_output 

print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.0+cu101 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [0]:
!pip install ipython-autotime
%load_ext autotime

# Environment setup

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from IPython.core.debugger import set_trace

from torchvision.utils import make_grid
from torchsummary import summary
from PIL import Image
# New 
from torchvision import models
from zipfile import ZipFile

time: 189 ms


In [0]:
from pathlib import Path
import os ,sys, gc ,tarfile ,zipfile,pickle

homepath = Path('/content/drive/My Drive/EVA4/S15')
libPath  = Path('/content/drive/My Drive/EVA4/S15/lib') 
#scripts  = Path('/content/drive/My Drive/EVA4/S15/scripts')
#dataset  = Path('/content/drive/My Drive/EVA4/S15/dataset/pklFiles/')
dataPath  = Path('/content/drive/My Drive/EVA4/S15/dataset/zipFiles')

sys.path.append(libPath)
os.chdir(homepath)

time: 4.65 ms


In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda
time: 1.84 ms


# Load dataset

## Transformations

In [0]:
import torchvision
import torchvision.transforms as transforms

def transformations():
    transform_train = transforms.Compose([
        # transforms.RandomCrop(32, padding=4),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ])

    return (transform_train, transform_test)

transform_train, transform_test = transformations()

time: 4.72 ms


# Custom dataset

In [0]:
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from PIL import Image
from zipfile import ZipFile

class customDataset(Dataset):
    def __init__(self, zipFName,transfrm):
        self.z_obj      = ZipFile(zipFName)
        self.transfrm   = transfrm
        # BG Images : dataPath.parent.joinpath('bg')
        self.bg         = os.listdir('/content/drive/My Drive/EVA4/S15/dataset/bg')

        # bgfg
        tmp             = list(filter(lambda x : x.startswith('bg_fg_1/'),self.z_obj.namelist()))
        tmp.remove('bg_fg_1/') if 'bg_fg_1/' in tmp else tmp
        self.bgfgF      = tmp
        del tmp
        # masks
        tmp             = list(filter(lambda x : x.startswith('bg_fg_mask_1/'),self.z_obj.namelist()))
        tmp.remove('bg_fg_mask_1/') if 'bg_fg_mask_1/' in tmp else tmp
        self.maskF      = tmp
        del tmp
        #depth
        tmp             =   list(filter(lambda x : x.startswith('depthMap/'),self.z_obj.namelist()))
        tmp.remove('depthMap/') if 'depthMap/' in tmp else tmp
        self.depthF     = tmp
        del tmp

    def __len__(self):
        return len(self.bgfgF)

    def __getitem__(self, idx):
        bgname = os.path.basename(self.bgfgF[idx]).split('_bg_')[0]+'_bg.jpg'
        bgF    = os.path.join('/content/drive/My Drive/EVA4/S15/dataset/bg',bgname)

        bgImg   = self.transfrm(Image.open(bgF))
        bgfgImg = self.transfrm(Image.open((self.z_obj.open(self.bgfgF[idx]))))
        maskImg = self.transfrm(Image.open((self.z_obj.open(self.maskF[idx]))))
        depthImg = self.transfrm(Image.open((self.z_obj.open(self.depthF[idx]))))

        return {'bgK' : bgImg,'bgfgK': bgfgImg, 'maskK' : maskImg, 'depthK': depthImg}

    def __del__(self):
        self.z_obj.close()
        del self.bgfgF

time: 38.7 ms


In [0]:
from torch.utils.data import DataLoader,random_split

trainset = customDataset(zipFName = r'/content/drive/My Drive/EVA4/S15/dataset/zipFiles/batch_1.zip' ,transfrm = transform_train)
train_set, test_set  = random_split(trainset,[int(0.7*len(trainset)),int(0.3*len(trainset))])

#trainloader = DataLoader(trainset, batch_size = 50, shuffle =True, pin_memory=True,num_workers =1)
trainloader = DataLoader(train_set, batch_size = 50, shuffle =True, pin_memory=True,num_workers =0)
testloader  = DataLoader(test_set, batch_size = 50, shuffle =True, pin_memory=True,num_workers =0)


time: 273 ms


In [0]:
print(len(train_set),len(test_set))

7000 3000
time: 2.51 ms


In [0]:
imgs = train_set.__getitem__(3)

time: 17.1 ms


In [0]:
print(type(imgs['bgK']))
print(type(imgs['bgfgK']))
print(type(imgs['maskK']))
print(type(imgs['depthK']))

del imgs

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
time: 2.29 ms


## Display few images from the dataset

In [0]:
import torchvision

def show(tensors, figsize=(100,100), *args, **kwargs):
    grid_tensor = torchvision.utils.make_grid(tensors[:8], *args, **kwargs)
    grid_image = grid_tensor.permute(1,2,0)
    plt.figure(figsize=figsize)
    plt.imshow(grid_image)
    plt.xticks([])
    plt.yticks([])
    plt.show()
    plt.close()


def saveplot(tensors, name, figsize=(100,100), *args, **kwargs):
	grid_tensor = torchvision.utils.make_grid(tensors[:8], *args, **kwargs)
	grid_image=grid_tensor.permute(1,2,0)
	plt.figure(figsize=figsize)
	plt.imshow((grid_image))
	plt.xticks([])
	plt.yticks([])
	plt.savefig(name, bbox_inches='tight')
	plt.close()

time: 13 ms


In [0]:
#sample = next(iter(trainloader))
#sample['maskK'].shape

time: 798 µs


In [0]:
#show(sample['bgK'])

time: 480 µs


In [0]:
#show(sample['bgfgK'])

time: 791 µs


In [0]:
#show(sample['maskK'])

time: 962 µs


In [0]:
#show(sample['depthK'])

time: 597 µs


In [0]:
#del sample

time: 451 µs


# Model

## Downsize

In [0]:
class Downsize (nn.Module):
    def __init__(self, inchannels, outchannels):
        super (Downsize,self).__init__()
        self.c1 = nn.Conv2d(inchannels, outchannels, kernel_size=3, stride=2, padding = 1, bias=False)
        self.conv11 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, bias=False)
        self.bn1=nn.BatchNorm2d(outchannels)
        self.relu=nn.ReLU()

        self.c2= nn.Conv2d(outchannels, outchannels, kernel_size=3, stride=2, padding = 1, bias=False)
        self.conv21 = nn.Conv2d(outchannels, outchannels, kernel_size=1, stride=1, bias=False)
        self.bn2=nn.BatchNorm2d(outchannels)

        self.dconv = nn.ConvTranspose2d(inchannels, outchannels,kernel_size=1, stride=2, bias=False)
    def forward(self,x):
        idchannel=self.dconv(x)
        x1=self.c1(x)
        x2=self.conv11(x1)
        x3=self.bn1(x2)
        x4=self.relu(x3)
        x5=self.c2(x4)
        x6=self.conv21(x5)
        x7=self.bn2(x6)
        x8=x7
        #x8=x7+idchannel
        x9=self.relu(x8)
        return x9


time: 36.2 ms


## upsize

In [0]:
class Upsize(nn.Module):
    def __init__(self,inchannels,outchannels):
        super(Upsize, self).__init__()
        self.c1=nn.Conv2d(outchannels, outchannels, kernel_size=3, padding=1,bias=False)
        self.conv11=nn.Conv2d(outchannels, outchannels, kernel_size=1, bias=False)
        self.bn1=nn.BatchNorm2d(outchannels)
        self.relu=nn.ReLU()
        self.c2=nn.Conv2d(outchannels, outchannels, kernel_size=3, padding=1,bias=False)
        self.conv21=nn.Conv2d(outchannels, outchannels, kernel_size=1, bias=False)
        self.bn2=nn.BatchNorm2d(outchannels)

        self.dconv=nn.ConvTranspose2d(inchannels, outchannels,kernel_size=3, padding=1, bias=False)

    def forward(self,x):
        idchannel=self.dconv(x)
        x1=self.c1(idchannel)
        x2=self.conv11(x1)
        x3=self.bn1(x2)
        x4=self.relu(x3)
        x5=self.c2(x4)
        x6=self.conv21(x5)
        x7=self.bn2(x6)
        x8=self.relu(x7)
        return x8
    

time: 21.6 ms


## Depthmask

In [0]:
class DepthMask(nn.Module):
    def __init__(self):
        super(DepthMask, self).__init__()

        self.c1=nn.Sequential(
            nn.Conv2d(6,64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.maxpool=nn.MaxPool2d(3, stride=2, padding=1)

        self.fdm1=Downsize(64,128)
        self.fdm2=Downsize(128,256)
        self.fdm3=Downsize(256,512)
        self.fdm4=Upsize(512,256)
        self.fdm5=Upsize(256,128)
        self.fdm6=Upsize(128,64)
        self.fdm7=Upsize(64,64)
        self.fdm8=Upsize(64,32)
        self.lasconv=nn.Sequential(
            nn.Conv2d(32,1,kernel_size=1, bias=False)
        )
        self.fdm4v=Upsize(512,256)
        self.fdm5v=Upsize(256,128)
        self.fdm6v=Upsize(128,64)
        self.fdm7v=Upsize(64,64)
        self.fdm8v=Upsize(64,32)
        self.lastconvd=nn.Sequential(
            nn.Conv2d(32,3,kernel_size=1, stride=1)
        )

    def forward (self, x):
   
        #z = torch.cat([sample['bg'], sample['bgfg']], dim=1)
        #z=torch.cat([x,y], dim=1)
        x0=self.c1(x)
        x1=self.maxpool(x0)
        x2=self.fdm1(x1)
        x3=self.fdm2(x2)
        x4=self.fdm3(x3)

        x4=nn.functional.interpolate(x4, scale_factor=4, mode='bilinear')
        x5=self.fdm4(x4)
        #print('x3:',x3.shape)
        #print('x4:',x4.shape)
        x5+=x3
        x5=nn.functional.interpolate(x5, scale_factor=7, mode='bilinear')
        x6=self.fdm5(x5)
        x2=nn.functional.interpolate(x2, scale_factor=2, mode='bilinear')
        #print('x6:',x6.shape)
        #print('x2:',x2.shape)
        
        x6+=x2
        x6=nn.functional.interpolate(x6, scale_factor=2, mode='bilinear')
        x7=self.fdm6(x6)
        x7+=x1
        x7=nn.functional.interpolate(x7, scale_factor=2, mode='bilinear')
        x8=self.fdm7(x7)
        x8+=x0
        x8=nn.functional.interpolate(x8, scale_factor=2, mode='bilinear')
        x9=self.fdm8(x8)
        x10=self.lasconv(x9)


        x5v=self.fdm4v(x4)
        x5v+=x3
        x5v=nn.functional.interpolate(x5v, scale_factor=7, mode='bilinear')
        x6v=self.fdm5(x5v)
        #print('x6v shape:',x6v.shape)
        #print('x2 shape:',x2.shape)
        x6v+=x2
        x6v=nn.functional.interpolate(x6v, scale_factor=2, mode='bilinear')
        x7v=self.fdm6(x6v)
        x7v=nn.functional.interpolate(x7v, scale_factor=2, mode='bilinear')
        x8v=self.fdm7(x7v)
        x8v+=x0
        x8v=nn.functional.interpolate(x8v, scale_factor=2, mode='bilinear')
        x9v=self.fdm8(x8v)
        x10Depth=self.lastconvd(x9v)
        return (x10, x10Depth)

time: 69.5 ms


## Loss nd optimizer

In [0]:
net = DepthMask()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

from torchsummary import summary
# Display Model Summary
model = net.to(device)
summary(model, input_size=(6, 224, 224))

criterion = nn.BCEWithLogitsLoss() #SSIM( 3, reduction ="mean")
optimizer = torch.optim.SGD(model.parameters(),lr= 0.01, momentum=0.9, weight_decay=1e-5)

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]          18,816
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
   ConvTranspose2d-5        [-1, 128, 111, 111]           8,192
            Conv2d-6          [-1, 128, 28, 28]          73,728
            Conv2d-7          [-1, 128, 28, 28]          16,384
       BatchNorm2d-8          [-1, 128, 28, 28]             256
              ReLU-9          [-1, 128, 28, 28]               0
           Conv2d-10          [-1, 128, 14, 14]         147,456
           Conv2d-11          [-1, 128, 14, 14]          16,384
      BatchNorm2d-12          [-1, 128, 14, 14]             256
             ReLU-13          [-1, 128, 14, 14]               0
         Downsize-14          [-1,

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


## train : Modified

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

from lib.modelSaveNLoad import *

def train(model,criterion,device,trainloader,optimizer,epochs):
    train_loss_mask, train_loss_depth = [],[]
    model.train()
    pbar = tqdm(trainloader)
    ### Train #####
    ################
    for batch_idx, data in enumerate(pbar):
        #print('Batch ID: ',batch_idx+1)
        gc.collect()
        optimizer.zero_grad()
        bg    = data['bgK'].to(device)
        bgfg  = data['bgfgK'].to(device)
        maskGt  = data['maskK'].to(device)
        depthGt = data['depthK'].to(device)
        ###########
        try:
            z=torch.cat([bgfg,bg], dim=1)
            (maskPred,depthPred) = model(z)  #(mask,depth)
        except RuntimeError as e:
            if 'CUDA out of memory' in str(e):
                print('| WARNING: ran out of memory, retrying batch',sys.stdout)
                print('Net parameters are : {}'.format(net.parameters()))
                #set_trace()
                sys.stdout.flush()
                for p in net.parameters():
                    if p.grad is not None:
                        del p.grad  # free some memory
                torch.cuda.empty_cache()
                (maskPred,depthPred) = model(z)
            else:
                raise e
        ############
        loss_mask = criterion(maskPred,maskGt)
        train_loss_mask.append(loss_mask)
        loss_depth =criterion(depthPred,depthGt)
        train_loss_depth.append(loss_depth)
        loss = 2*loss_mask+loss_depth        
        loss_min = loss
        pbar.set_description(desc= "Overall Loss={:.4f} Mask loss={:.2f} Depth loss={:.2f}".format(loss.item(),loss_mask.item(), loss_depth.item()))
        loss.backward()
        optimizer.step()
        ###########
        ### Delete scope variables
        del bg, bgfg,maskGt,depthGt
        
        if batch_idx %50 == 0:
            print('batch_idx: {}'.format(batch_idx))
            print('loss: {}'.format(loss))
            print('Train Epoch: {}  [{}/{}  ({:.0f}%)]\tLoss:{:.6f}'.format(
                epochs,batch_idx*len(data),len(trainloader.dataset),
                100.*batch_idx/len(trainloader),loss.item()))
            show(maskPred.detach().cpu())
            show(depthPred.detach().cpu())
            del maskPred,depthPred
    print('After compleation of training at epoch :{}'.format(batch_idx))
    show(maskPred.detach().cpu())
    show(depthPred.detach().cpu())
    del maskPred,depthPred
    # create checkpoint variable and add important data
    checkpoint = {
        'epoch': epochs,
        'valid_loss_min': loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict(),
    }

    return checkpoint
    '''
    # save checkpoint
    save_ckp(checkpoint, False, checkpoint_path='batch_1_chkp.pt', best_model_path='batch_1_best.pt')
    ## TODO: save the model if validation loss has decreased
    if loss <= loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path='batch_1_chkp.pt', best_model_path='batch_1_best.pt')
        loss_min = loss
    '''
        
def test(model,criterion,device,testloader,epochs):
    test_loss_mask, test_loss_depth,test_loss = [],[],[]
    model.eval()
    pbar = tqdm(testloader)
    for batch_idx, data in enumerate(pbar):
        with torch.no_grad():
            #print('Batch ID: ',batch_idx+1)
            gc.collect()
            #optimizer.zero_grad()
            bg      = data['bgK'].to(device)
            bgfg    = data['bgfgK'].to(device)
            maskGt  = data['maskK'].to(device)
            depthGt = data['depthK'].to(device)
            ###########
            try:
                z=torch.cat([bgfg,bg], dim=1)
                (maskPred,depthPred) = model(z)  #(mask,depth)
            except RuntimeError as e:
                if 'CUDA out of memory' in str(e):
                    print('| WARNING: ran out of memory, retrying batch',sys.stdout)
                    print('Net parameters are : {}'.format(net.parameters()))
                    #set_trace()
                    sys.stdout.flush()
                    for p in net.parameters():
                        if p.grad is not None:
                            del p.grad  # free some memory
                    torch.cuda.empty_cache()
                    (maskPred,depthPred) = model(z)
                else:
                    raise e
            ############
            loss_mask = criterion(maskPred,maskGt).item()
            test_loss_mask.append(loss_mask)
            loss_depth =criterion(depthPred,depthGt).item()
            test_loss_depth.append(loss_depth)
            loss = 2*loss_mask+loss_depth 
            test_loss.append(loss)       
            #pbar.set_description(desc= "Overall Loss={:.4f} Mask loss={:.2f} Depth loss={:.2f}".format(loss.item(),loss_mask.item(), loss_depth.item()))
            #pbar.set_description(desc= "Overall Loss={:.4f} Mask loss={:.2f} Depth loss={:.2f}".format(loss.item(),loss_mask.item(), loss_depth.item()))
            if batch_idx %50 == 0:
                print('batch_idx: {}'.format(batch_idx))
                print('loss: {}'.format(loss))
                print('Train Epoch: {}  [{}/{}  ({:.0f}%)]\tLoss:{:.6f}'.format(
                    epochs,batch_idx*len(data),len(trainloader.dataset),
                    100.*batch_idx/len(trainloader),loss.item()))
                show(maskPred.detach().cpu())
                show(depthPred.detach().cpu())
       
                i = batch_idx
                print('inside save plot')
                saveplot(maskPred.detach().cpu(),'/content/'+str(epoch)+'_'+str()+'_predicted_mask.jpg')
                saveplot(maskGt.detach().cpu(),'/content/'+str(epoch)+'_'+str(i)+'_actual_mask.jpg')
                saveplot(depthPred.detach().cpu(),'/content/'+str(epoch)+'_'+str(i)+'_predicted_depth.jpg')
                saveplot(depthGt.detach().cpu(),'/content/'+str(epoch)+'_'+str(i)+'_actual_depth.jpg')
                
                del bg, bgfg,maskGt,depthGt
    #set_trace()
    print('\nTest set: Avg loss: {:.4f}, Mask Loss: {:.2f}, Depth Loss: {:.2f}\n'.format(np.mean(test_loss), np.mean(test_loss_mask), np.mean(test_loss_depth)))
    print('After compleation of Test at epoch :{}'.format(batch_idx))
    show(maskPred.detach().cpu())
    show(depthPred.detach().cpu())
    del maskPred,depthPred

time: 164 ms


In [0]:
from lib.modelSaveNLoad import *

EPOCHS = 5
loss = 1e+5
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    checkpoint = train(model, criterion, device, trainloader,optimizer, epochs=epoch)
    #save_ckp(checkpoint, False, checkpoint_path='batch_1_chkp.pt', best_model_path='batch_1_best.pt')
    ## TODO: save the model if validation loss has decreased
    #set_trace()
    if checkpoint['valid_loss_min'].item() < loss:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(loss,checkpoint['valid_loss_min'].item()))
        # save checkpoint as best model
        torch.save(checkpoint,r'/content/drive/My Drive/EVA4/S15/modelWeights/checkpoint/batch_best.ckp.pt')
        #save_ckp(checkpoint, True, checkpoint_path='batch_1_chkp.pt', best_model_path='batch_1_best.pt')
        loss = checkpoint['valid_loss_min'].item()
    test(model,criterion,device,testloader,epoch)

    

Output hidden; open in https://colab.research.google.com to view.